# Import Libraries

In [32]:
import datetime
from dateutil.relativedelta import relativedelta
import os
import requests
from github import Github
import pandas as pd
from bs4 import BeautifulSoup

# Trending technologie

In [30]:
def trend_repositories_names(since='daily'):
    #Build request URL
    url=f"https://github.com/trending?since={since}"
    #send request and parse HTML to exc data
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    titles = soup.find_all('h2', class_='h3 lh-condensed')
    
    #Get the list of Trending repository
    trend_repository = []
    for title in titles:
        links = title.find_all('a')
        for link in links:
            href_value = link.get('href')
            trend_repository.append(href_value[1:])
    return trend_repository

trend_repo=trend_repositories_names(since='daily')

print(trend_repo)
print(f'Number of repositories: {len(trend_repo)}')

['AntonOsika/gpt-engineer', 'AI4Finance-Foundation/FinGPT', 'alibaba/Chat2DB', 'openai/openai-cookbook', 'stefan-jansen/machine-learning-for-trading', 'facebookresearch/ijepa', 'Victorwz/LongMem', 'hwchase17/langchain', 'sunner/ChatALL', 'hiyouga/LLaMA-Efficient-Tuning', 'YimMenu/YimMenu', 'krzysztofzablocki/Swift-Macros', 'ordinals/ord', 'tatsu-lab/alpaca_eval', 'imClumsyPanda/langchain-ChatGLM', 'gopherchina/conference', 'lm-sys/FastChat', 'microsoft/DeepSpeed', 'nrwl/nx', 'ataradov/usb-sniffer', 'open-mmlab/mmagic', 'imartinez/privateGPT', 'rvaidun/befake', 'Uniswap/v4-core', 'ziglang/zig']
Number of repositories: 25


In [31]:
token="YOUR TOKEN"

#method to get info about Trend repositories
def repositories_info(repositories_names):
    g = Github(token)
    trend_repo=[]
    for repo_name in repositories_names:
        repository=g.get_repo(repo_name)
        trend_repo.append({'Created at':repository.created_at,'name':repository.full_name,'languages':repository.language,
                           'description':repository.description,'topics':repository.topics,'Stars':repository.stargazers_count,
                           'Watchers':repository.watchers,'Forks':repository.get_forks().totalCount,'Issues':len(list(repository.get_issues())),
                           'Contributors':len(list(repository.get_contributors()))})
    return trend_repo

#pass liste of trend repositories to trend_repositories_info() to get infromation about repository
data=repositories_info(trend_repositories_names(since='daily'))

#Save trend repo information to CSV file
df = pd.DataFrame(data)
df.to_csv('trend_repo.csv')

BadCredentialsException: 401 {"message": "Bad credentials", "documentation_url": "https://docs.github.com/rest"}

# Popular technolige

In [ ]:
token="YOUR TOKEN"

def year_data(year,month):

    url = 'https://api.github.com/search/repositories'
    headers = {
                'Accept': 'application/vnd.github+json',
                'Authorization': f'Bearer {token}',
                'X-GitHub-Api-Version': '2022-11-28'
            }

    repos=[]

    start_date = datetime.date(year, month, 1)
    end_date = start_date+relativedelta(months=1)
    current_date = start_date

    while current_date < end_date and current_date <= datetime.date.today():
        #get 100 repo from page 1 to 10 and add him to repos liste
        for page in range (1,11):
            params = {'q': f"created:{current_date}",'per_page':100,'page':page,'sort':'stars, forks, help-wanted-issues, updated','order':'desc'}
            response = requests.get(url, headers=headers, params=params,timeout=60000)
            if(response.status_code==200):
                data = response.json()
                for repo in data['items']:
                    repos.append([repo["created_at"],repo["full_name"],repo["language"],repo["description"], 
                                  repo["topics"],repo["stargazers_count"],repo["watchers_count"],repo["forks_count"],
                                  repo["open_issues_count"],repo["html_url"]])
            else:
                raise Exception(f"Something worng! error code: {response.status_code}")
        current_date += datetime.timedelta(days=1)
    return repos


In [ ]:
#Save Data of one Month(do it for each month)
repos = year_data(2022,6)
columns = ["created_at", "full_name", "language", "description", "topics", "stargazers_count", "watchers_count",
            "forks_count", "open_issues_count", "html_url"]
df = pd.DataFrame(repos, columns=columns)
df.to_csv('Data/2022-06.csv')

# Merg Data

In [ ]:
# Specify the folder path where the CSV files are located
folder_path = 'PATH OF CSV FILES'

# Get the list of CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Initialize an empty list to store the DataFrames
dfs = []

# Iterate over each CSV file
for file in csv_files:
    # Read the CSV file into a DataFrame
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    
    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate the list of DataFrames into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

merged_df.to_csv('Data/2023-06-14..2022-06-01.csv', index=False)


In [28]:
df=pd.read_csv('Data/2023-06-14..2022-06-01.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378996 entries, 0 to 378995
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Unnamed: 0         378996 non-null  int64 
 1   created_at         378996 non-null  object
 2   full_name          378996 non-null  object
 3   language           333284 non-null  object
 4   description        272337 non-null  object
 5   topics             378996 non-null  object
 6   stargazers_count   378996 non-null  int64 
 7   watchers_count     378996 non-null  int64 
 8   forks_count        378996 non-null  int64 
 9   open_issues_count  378996 non-null  int64 
 10  html_url           378996 non-null  object
dtypes: int64(5), object(6)
memory usage: 31.8+ MB
